<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 820, done.
remote: Counting objects: 100% (372/372), done.
remote: Compressing objects: 100% (363/363), done.
remote: Total 820 (delta 33), reused 334 (delta 9), pack-reused 448
Receiving objects: 100% (820/820), 139.08 MiB | 19.87 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Checking out files: 100% (662/662), done.


In [113]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด',
               'ฝรั่ง',
               'พุทรา',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [114]:
import pandas as pd
from glob import glob
import librosa

for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [116]:
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
1,"[1.5258789e-05, -3.0517578e-05, 4.5776367e-05,...",22050,"[[-624.1429, -624.4267, -624.8286, -624.58374,...",0,กระชาย
2,"[-0.004928589, -0.0049591064, -0.0048980713, -...",22050,"[[-634.622, -639.0867, -654.00525, -659.99115,...",0,กระชาย
3,"[1.5258789e-05, -3.0517578e-05, 6.1035156e-05,...",22050,"[[-635.4628, -636.8742, -637.42847, -636.0504,...",0,กระชาย
4,"[-0.0001373291, -0.00018310547, -4.5776367e-05...",22050,"[[-655.651, -653.5817, -654.66907, -655.0482, ...",0,กระชาย
...,...,...,...,...,...
655,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-662.2502, -662.54706, -661.8858, -660.38995...",32,สับปะรด
656,"[-0.00019836426, -0.0002593994, -7.6293945e-05...",22050,"[[-655.92065, -656.85034, -655.4548, -653.7831...",32,สับปะรด
657,"[-0.00016784668, -0.00015258789, -0.0001983642...",22050,"[[-668.4974, -670.4946, -672.10724, -671.17737...",32,สับปะรด
658,"[-0.00018310547, -0.0002593994, -0.00021362305...",22050,"[[-670.37274, -672.4899, -674.93616, -677.3624...",32,สับปะรด


In [134]:
import numpy as np
import pandas as pd

Test = pd.DataFrame(columns=source.keys())

# create Test model
for description in np.unique(df['description']):
    imp = df[df['description']==description].sample(5,random_state=1)
    Test = pd.concat([Test,imp], axis=0)

# create Train model
Train = df.drop(Test.index)

X_train, X_test = np.array(Train['feature_extraction'].to_list()), np.array(Test['feature_extraction'].to_list())
y_train, y_test = np.array(Train['label'].to_list()), np.array(Test['label'].to_list())

In [135]:
import tensorflow as tf
from tensorflow import keras 

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
16/16 [==============================] - 1s 23ms/step - loss: 28.6359 - accuracy: 0.0263
Epoch 2/100
16/16 [==============================] - 0s 20ms/step - loss: 3.4892 - accuracy: 0.0485
Epoch 3/100
16/16 [==============================] - 0s 18ms/step - loss: 3.4722 - accuracy: 0.0444
Epoch 4/100
16/16 [==============================] - 0s 18ms/step - loss: 3.4521 - accuracy: 0.0384
Epoch 5/100
16/16 [==============================] - 0s 18ms/step - loss: 3.4206 - accuracy: 0.0404
Epoch 6/100
16/16 [==============================] - 0s 17ms/step - loss: 3.3786 - accuracy: 0.0505
Epoch 7/100
16/16 [==============================] - 0s 17ms/step - loss: 3.4074 - accuracy: 0.0404
Epoch 8/100
16/16 [==============================] - 0s 19ms/step - loss: 3.3794 - accuracy: 0.0404
Epoch 9/100
16/16 [==============================] - 0s 18ms/step - loss: 3.3550 - accuracy: 0.0465
Epoch 10/100
16/16 [==============================] - 0s 18ms/step - loss: 3.3076 - accuracy: 0.056

In [136]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 9, 255, 64)        1664      
_________________________________________________________________
dropout_67 (Dropout)         (None, 9, 255, 64)        0         
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 4, 127, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 4, 127, 64)        102464    
_________________________________________________________________
dropout_68 (Dropout)         (None, 4, 127, 64)        0         
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 2, 63, 64)         0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 8064)            

In [137]:
model.evaluate(X_test[:,:,:,None],y_test)

6/6 [==============================] - 0s 9ms/step - loss: 0.3254 - accuracy: 0.9515


[0.32544514536857605, 0.9515151381492615]

In [138]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = ["Class {}".format(i) for i in range(len(np.unique(source['label'])))]
print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.62      1.00      0.77         5
     Class 1       1.00      1.00      1.00         5
     Class 2       1.00      1.00      1.00         5
     Class 3       1.00      0.80      0.89         5
     Class 4       1.00      1.00      1.00         5
     Class 5       1.00      1.00      1.00         5
     Class 6       1.00      1.00      1.00         5
     Class 7       1.00      1.00      1.00         5
     Class 8       1.00      1.00      1.00         5
     Class 9       1.00      1.00      1.00         5
    Class 10       1.00      1.00      1.00         5
    Class 11       1.00      1.00      1.00         5
    Class 12       1.00      1.00      1.00         5
    Class 13       1.00      0.80      0.89         5
    Class 14       1.00      1.00      1.00         5
    Class 15       0.75      0.60      0.67         5
    Class 16       0.83      1.00      0.91         5
    Class 17       0.83    